# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
! jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
! pip install gmaps

  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076101 sha256=13d803e9ded18815db03207c269fdcba4fd303b2a339615cfc60a8a83138006b
  Stored in directory: C:\Users\kandr\AppData\Local\pip\Cache\wheels\33\eb\41\5302168c36fc8ce8f48143bb58fde1015d5bc0c62bc3ddbb38
Successfully built gmaps


In [4]:
! jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

weather_df=pd.read_csv('./weather_data_KR.csv')
weather_df

,Unnamed: 0,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,0,6170031,tuktoyaktuk,CA,69.45,-133.04,1590173685,37.40,74,5,6.93
1,1,3018794,fecamp,FR,49.76,0.37,1590173685,57.25,82,90,9.17
2,2,623760,pastavy,BY,55.12,26.83,1590173686,42.60,70,0,8.14
3,3,4030556,rikitea,PF,-23.12,-134.97,1590173686,71.11,69,94,19.57
4,4,1507390,dikson,RU,73.51,80.55,1590173531,40.15,100,100,9.66
...,...,...,...,...,...,...,...,...,...,...,...
575,575,1735076,butterworth,MY,5.40,100.36,1590173735,80.31,88,20,6.93
576,576,364933,umm kaddadah,SD,13.60,26.69,1590173735,94.86,24,0,10.25
577,577,5935341,dauphin,CA,51.15,-100.05,1590173735,78.80,42,1,11.41
578,578,3661894,tonantins,BR,-2.87,-67.80,1590173735,87.39,69,57,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Store latitude and longitude in locations
locations = weather_df[["latitude", "longitude"]]

# Fill NaN values and convert to float
hum_weight = weather_df["humidity"]
#locations
#hum_weight

0       74
1       82
2       70
3       69
4      100
      ... 
575     88
576     24
577     42
578     69
579     86
Name: humidity, Length: 580, dtype: int64

In [11]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

Observing the humidty level near the Gulf of Mexico, I would expect to see higher humidity. I expected to see high humidity over more of Australia not just the southern portion. Also, I am surprised Iceland's humidity can get high.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.temp < 80) & (weather_df.temp > 70)
crit_windSpeed = weather_df.wind_speed < 15
crit_cloudiness = weather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,Unnamed: 0,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,8,8,3170069,porto torres,IT,40.84,8.40,1590173686,71.62,46,0,10.29
1,45,45,1160571,khash,IR,28.22,61.22,1590173690,72.00,35,0,1.19
2,83,83,3374210,sao filipe,CV,14.90,-24.50,1590173693,75.99,69,0,8.63
3,89,89,6160362,swan river,CA,52.11,-101.27,1590173693,74.35,36,0,4.85
4,125,125,1274642,channagiri,IN,14.03,75.93,1590173696,76.46,65,0,13.56
5,195,195,3473218,acajutiba,BR,-11.66,-38.02,1590173702,78.44,84,0,8.75
6,232,232,3466980,caravelas,BR,-17.71,-39.25,1590173705,78.53,74,0,5.82
7,246,246,1142226,dawlatabad,AF,36.41,64.91,1590173706,77.07,22,0,1.52
8,358,358,3347019,namibe,AO,-15.20,12.15,1590173715,76.59,63,0,9.42
9,365,365,608872,makat,KZ,47.65,53.35,1590173716,76.62,36,0,3.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,Unnamed: 0,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed,Hotel Name
0,8,8,3170069,porto torres,IT,40.84,8.40,1590173686,71.62,46,0,10.29,
1,45,45,1160571,khash,IR,28.22,61.22,1590173690,72.00,35,0,1.19,
2,83,83,3374210,sao filipe,CV,14.90,-24.50,1590173693,75.99,69,0,8.63,
3,89,89,6160362,swan river,CA,52.11,-101.27,1590173693,74.35,36,0,4.85,
4,125,125,1274642,channagiri,IN,14.03,75.93,1590173696,76.46,65,0,13.56,


In [15]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city_name</dt><dd>{city_name}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [22]:
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…